In [5]:
# This script is intended to download all the 
# txt UniProt files for the kinases proteins 
# and search the calatytic sites

In [1]:
import requests
import pandas as pd
import os
import re

In [11]:
# Set Leloir proxy for downloads
proxies = {
    'http': 'http://proxy.leloir.org.ar:3128',
    'https': 'http://proxy.leloir.org.ar:3128'
}

### Download the uniprot files for all the kinases

In [6]:
# All kinases dataset
kinases = pd.read_csv('../datasets/kinases.tsv', sep= '\t')
kinases.columns = kinases.columns.str.lower().str.replace(" ", "_")

In [3]:
# Table with pdb ids of kinases containing CREs dataset
pdbs_kd_cre = pd.read_csv('../datasets/pdbs_kd_cre.tsv', sep= '\t')
pdbs_kd_cre

,uniprot_acc,id_pdb,id_cre,id_kd,pdb
0,A5K0N4,5147,10,10,5fet
1,O74536,8034,21,21,3h4j
2,O74536,8035,21,21,3h4j
3,O75582,579,22,23,3kn5
4,O75582,582,22,23,3kn5
...,...,...,...,...,...
1532,Q16644,2845,142,147,3fhr
1533,Q16644,5339,142,147,7nrb
1534,Q61846,1754,178,183,4bfm
1535,Q61846,7063,178,183,4cqg


In [9]:
uniprot_ids = kinases.uniprot.unique().tolist()

In [13]:
# download uniprot txt file for each protein
error = {}
already = []
for uniprot in uniprot_ids:
    r = requests.get("https://rest.uniprot.org/uniprotkb/" + uniprot + ".txt", proxies= proxies)
    if r.status_code == 200:
        text_resp = r.text

        f = open("../datasets/uniprot_files/" + uniprot + ".txt", "w")
        f.write(text_resp)
        f.close()

        already.append(uniprot)
        
    else:
        error[uniprot] = r.status_code

In [14]:
error # None. cool

{}

In [28]:
# # esto no
# path = '../datasets/uniprot_files/'

# for filename in os.listdir(path):
#     f = os.path.join(path, filename)
#     if os.path.isfile(f):
#         #print(f)
#         # string to search in file
#         with open(f, 'r') as fp:
#             # read all lines using readline()
#             lines = fp.readlines()
#             for row in lines:
#                 # check if string present on a current line
#                 word = 'FT   ACT_SITE'
#         #         print(row.find(word))
#                 # find() method returns -1 if the value is not found,
#                 # if found it return 0
#                 if row.find(word) == 0:
#                     print('string exists in file')
#                     #print('line Number:', lines.index(line))

In [15]:
# Find the term ACT_SITE in the files
path = '../datasets/uniprot_files/'
pos_global = []

# iterate over the uniprot files
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):

        # string to search in file
        with open(f, 'r') as fp:
            # read all lines using readline()
            lines = fp.readlines()
            filename = os.path.basename(filename)
            filename = os.path.splitext(filename)[0]
            for i, row in enumerate(lines):
                # check if string present on a current line
                pattern = "^FT   ACT_SITE        (.+)$"
                k = re.findall(pattern, row)
                if k:
                    pos_global.append({'uniprot': filename, 'pos': k[0]})
                #print(row)
                #print(pos)
                # if row.find(word) == 0:
                #     print('string exists in file')
                #     #print('line Number:', lines.index(line))          

In [16]:
catalytic_sites = pd.DataFrame.from_dict(pos_global)#.reset_index()
catalytic_sites

,uniprot,pos
0,A0A2I0BVG8,191
1,A0A509AFG4,230
2,A0A509AHB6,190
3,A0A509AKL0,668
4,A0A509AQE6,238
...,...,...
278,Q9ZSA4,156
279,Q9ZUZ2,271
280,Q9ZV15,258
281,W0LYS5,153


In [17]:
catalytic_sites.uniprot.nunique()

278

In [18]:
#catalytic_sites.to_csv('../datasets/cat_sites_uniprot.tsv', sep='\t', index= False)

---

In [59]:
# For the protein domains
path = '../datasets/uniprot_files/'
domain_global = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        # string to search in file
        with open(f, 'r') as fp:
            # read all lines using readline()
            lines = fp.readlines()
            filename = os.path.basename(filename)
            filename = os.path.splitext(filename)[0]
            i = 0
            pattern1 = "^FT   DOMAIN          (\d+)\.?\.?(\d+)?$"
            pattern2 = "FT                   \/(.*)$"
            while i < len(lines):
                k = re.findall(pattern1, lines[i])
                data = []
                if k:
                    while True:
                        i += 1
                        m = re.findall(pattern2, lines[i])
                        if m:
                            data.append(m[0])
                        else:
                            break
                    domain_global.append({'uniprot': filename, 'start': k[0][0], 'end': k[0][1], 'metadata': data})
                else:
                    i += 1

In [61]:
domain_global

[{'uniprot': 'A5K0N4',
  'start': '534',
  'end': '791',
  'metadata': ['note="Protein kinase"',
   'evidence="ECO:0000255|PROSITE-ProRule:PRU00159"']},
 {'uniprot': 'A5K0N4',
  'start': '792',
  'end': '846',
  'metadata': ['note="AGC-kinase C-terminal"',
   'evidence="ECO:0000255|PROSITE-ProRule:PRU00618"']},
 {'uniprot': 'O74536',
  'start': '34',
  'end': '285',
  'metadata': ['note="Protein kinase"',
   'evidence="ECO:0000255|PROSITE-ProRule:PRU00159,']},
 {'uniprot': 'O74536',
  'start': '304',
  'end': '345',
  'metadata': ['note="UBA"',
   'evidence="ECO:0000255|PROSITE-ProRule:PRU00212"']},
 {'uniprot': 'O75582',
  'start': '49',
  'end': '318',
  'metadata': ['note="Protein kinase 1"',
   'evidence="ECO:0000255|PROSITE-ProRule:PRU00159"']},
 {'uniprot': 'O75582',
  'start': '319',
  'end': '387',
  'metadata': ['note="AGC-kinase C-terminal"',
   'evidence="ECO:0000255|PROSITE-ProRule:PRU00618"']},
 {'uniprot': 'O75582',
  'start': '426',
  'end': '687',
  'metadata': ['note="

In [60]:
domains = pd.DataFrame.from_dict(domain_global)
domains

,uniprot,start,end,metadata
0,A5K0N4,534,791,"[note=""Protein kinase"", evidence=""ECO:0000255|..."
1,A5K0N4,792,846,"[note=""AGC-kinase C-terminal"", evidence=""ECO:0..."
2,O74536,34,285,"[note=""Protein kinase"", evidence=""ECO:0000255|..."
3,O74536,304,345,"[note=""UBA"", evidence=""ECO:0000255|PROSITE-Pro..."
4,O75582,49,318,"[note=""Protein kinase 1"", evidence=""ECO:000025..."
...,...,...,...,...
87,Q6PHZ2,14,272,"[note=""Protein kinase"", evidence=""ECO:0000255|..."
88,Q8IU85,23,279,"[note=""Protein kinase"", evidence=""ECO:0000255|..."
89,Q8VDF3,23,285,"[note=""Protein kinase"", evidence=""ECO:0000255|..."
90,Q96NX5,23,277,"[note=""Protein kinase"", evidence=""ECO:0000255|..."


In [ ]:
domains["start"] = domains

---

In [63]:
kinases = pd.read_csv('../datasets/kinases.tsv', sep= '\t')

In [65]:
kinases[kinases.Uniprot == 'O75582']

,Uniprot,Organism,CRE (start),CRE (end),CRE (S or D),Length,KD (start),KD (end),PDB ID,PDB Res (Å),AF,Status
22,O75582,H. sapiens,702,723,S,NaN,49,318,1VZO,1.80,AF-O75582-F1,known
23,O75582,H. sapiens,702,723,S,NaN,426,687,3KN6,2.00,AF-O75582-F1,known


In [20]:
# # string to search in file
# with open('../datasets/uniprot_files/A5K0N4.txt', 'r') as fp:
#     # read all lines using readline()
#     lines = fp.readlines()
#     for row in lines:
#         # check if string present on a current line
#         word = 'ACT_SITE'
# #         print(row.find(word))
#         # find() method returns -1 if the value is not found,
#         # if found it return 0
#         if row.find(word) == 0:
#             print('string exists in file')
#             #print('line Number:', lines.index(line))

None of them contain the ACTIVE_SITE text